In [6]:
import sys
sys.path.append(r"d:\Università\terzo anno\Tesi\astrocook")
from astrocook.functions import lines_voigt, convolve_simple
from astrocook import vars
from astropy.table import Table
import numpy as np
import scipy.signal as sps
from scipy.interpolate import interp1d
from tqdm import tqdm


In [31]:

##FUNCTIONS##

# Get indices function
def get_indicies(spec_chunk, flux, threshold, i, ion, z):
    if i == 0:
        ion_components = {key: value for key, value in vars.xem_d.items() if ion in key}
        c1 = float(ion_components[list(ion_components.items())[0][0]].value)
        c2 = float(ion_components[list(ion_components.items())[1][0]].value)
        mask = ((spec_chunk['wavelength'] > c1 *(1+z)) & (spec_chunk['wavelength'] < c2*(1+z))) | (flux < 1- threshold)

        indicies = np.where(mask)[0]
    else:
        indicies = np.where((flux < 1- threshold))[0]
    
    count = len(indicies)
        
    return indicies, count


In [32]:

# Function to convert bin to redshift
def bin_to_z(bin, z_start, dz):
    return z_start + bin * dz


In [27]:

def psf_gauss(x, resol):
        if len(x)==0:
            return []
        c = x[len(x)//2]
        sigma = c / resol * 4.246609001e-1

        psf = np.exp(-0.5*((x-c) / sigma)**2)
        psf = psf[np.where(psf > 1e-6)]
        return psf


In [28]:
spectrum_file = r'D:\Università\terzo anno\Tesi\AC_Correlator\test_mos_spec.dat'
resol = 45000
wav_start = [154.8, 154.8, 155.06]
wav_end = [155.1, 154.85, 155.1]
logN = 12
b = 5
btur = 0
ion = 'CIV'
threshold_value = 0.999
dz = 1e-5
perc = 75

In [29]:

# Load spectrum
spectrum = Table.read(spectrum_file, format='ascii')
threshold = np.std(spectrum['dy'])

spectrum = Table([spectrum['x'], spectrum['y']], names=['wavelength', 'flux'])
    
# Define models
models = [Table(), Table(), Table()]
for i in range(len(wav_start)):
    x = np.linspace(wav_start[i], wav_end[i], 1000)
    y = convolve_simple(lines_voigt(x, 0, logN, b, btur, ion), psf_gauss(x, resol))
    models[i] = Table([x, y], names=['wavelength', 'flux'])


In [33]:

# Other parameters
z_start = spectrum['wavelength'][0] / 154.8 - 1
z_end = spectrum['wavelength'][-1] / 155.1 - 1

cor_all = [np.array([]), np.array([]), np.array([])]

for i, model in enumerate(models):
    for z in tqdm(np.arange(z_start, z_end, dz), f"Calculating Correlation with model {i}"): 
        # Defining the interval in which the model is present
        interval = [(model['wavelength'].min())*(1+z), (model['wavelength'].max())*(1+z)]    
        mask = (spectrum['wavelength'] > interval[0]) & (spectrum['wavelength'] < interval[1])

        # Selecting the data interval covered by the model
        spec_chunk = spectrum[mask]

        # Interpolating the model to the data wavelength
        interpolate = interp1d(model['wavelength']*(1 + z), model['flux'], kind='linear')
        interpolated_flux = interpolate(spec_chunk['wavelength'])

        # Identifying the indices of the model that are below the threshold
        indicies, count = get_indicies(spec_chunk, interpolated_flux, threshold, ion, i, z)

        cor = np.correlate(interpolated_flux[indicies], spec_chunk['flux'][indicies], mode='valid') / count
        cor_all[i] = np.append(cor_all[i], cor)

# Percentile-based thresholding
p0 = np.percentile(cor_all[0], perc)
p1 = np.percentile(cor_all[1], perc)
p2 = np.percentile(cor_all[2], perc)

cor_final = (p0 - cor_all[0]) * (p1 - cor_all[1]) * (p2 - cor_all[2])
z_interval = np.arange(z_start, z_end, dz)

# Finding the peaks
peaks, properties = sps.find_peaks(cor_final, height=np.mean(cor_final) + np.std(cor_final) * 2.5, prominence=0, width=0.01, distance=5e-4 / dz)
peaks_table = Table([bin_to_z(peaks, z_start, dz), properties['peak_heights'], properties['widths'], bin_to_z(properties['left_ips'], z_start, dz), bin_to_z(properties['right_ips'], z_start, dz), properties['width_heights'], properties['prominences']],
                    names=['z', 'height', 'fwhm', 'left_z', 'right_z', 'half_max', 'prominence'])


Calculating Correlation with model 2: 100%|██████████| 52965/52965 [01:14<00:00, 713.83it/s] 
